# iMaterialist_Kaggle_Competition

https://www.kaggle.com/c/imaterialist-challenge-fashion-2018

In [3]:
import sys, os, multiprocessing, urllib3, csv
from PIL import Image
from io import BytesIO
import pandas as pd

Dataset is provided as a list of urls. First step is to get all the images.

#### Downloading validation and testing image sets

In [1]:
PATH = "/media/james/HDD/Data_sets/iMaterialist/jsonfiles/"

In [55]:
df = pd.read_json(f"{PATH}validation.json")

In [6]:
def anno_checker(anno):
    known = ['labelId', 'imageId']
    for key in anno.keys():
        if key not in known: 
            return 1

In [7]:
df.annotations.apply(anno_checker).sum() #annotations only contain ids and labels

0

In [8]:
def anno_puller(anno):
    return [int(label) for label in anno['labelId']]

In [9]:
df['annos'] = df.annotations.apply(anno_puller) #sort nested dicts to columns
df['annos_id'] = df.annotations.apply(lambda x: x['imageId'])
df['image_id'] = df.images.apply(lambda x: x['imageId'])
df['image_url'] = df.images.apply(lambda x: x['url'])

In [10]:
df.head()

,annotations,images,annos,annos_id,image_id,image_url
0,"{'labelId': ['62', '17', '66', '214', '105', '...",{'url': 'https://contestimg.wish.com/api/webim...,"[62, 17, 66, 214, 105, 137, 85]",1,1,https://contestimg.wish.com/api/webimage/568e1...
1,"{'labelId': ['95', '17', '66', '214', '164', '...",{'url': 'https://contestimg.wish.com/api/webim...,"[95, 17, 66, 214, 164, 137, 20, 204, 184]",2,2,https://contestimg.wish.com/api/webimage/5452f...
2,"{'labelId': ['122', '19', '66', '186', '180', ...",{'url': 'https://contestimg.wish.com/api/webim...,"[122, 19, 66, 186, 180, 44, 154, 20]",3,3,https://contestimg.wish.com/api/webimage/54058...
3,"{'labelId': ['190', '222', '66', '153', '164',...",{'url': 'https://contestimg.wish.com/api/webim...,"[190, 222, 66, 153, 164, 226, 53, 184]",4,4,https://contestimg.wish.com/api/webimage/540c6...
4,"{'labelId': ['62', '66', '153', '171', '111', ...",{'url': 'https://contestimg.wish.com/api/webim...,"[62, 66, 153, 171, 111, 137, 70, 204, 184]",5,5,https://contestimg.wish.com/api/webimage/54477...


In [11]:
(df.annos_id == df.image_id).all() #all image urls and labels match

True

In [12]:
df.drop(['annotations','images','annos_id'], axis=1, inplace=True)

In [14]:
df.head()

,annos,image_id,image_url
0,"[62, 17, 66, 214, 105, 137, 85]",1,https://contestimg.wish.com/api/webimage/568e1...
1,"[95, 17, 66, 214, 164, 137, 20, 204, 184]",2,https://contestimg.wish.com/api/webimage/5452f...
2,"[122, 19, 66, 186, 180, 44, 154, 20]",3,https://contestimg.wish.com/api/webimage/54058...
3,"[190, 222, 66, 153, 164, 226, 53, 184]",4,https://contestimg.wish.com/api/webimage/540c6...
4,"[62, 66, 153, 171, 111, 137, 70, 204, 184]",5,https://contestimg.wish.com/api/webimage/54477...


In [49]:
labelcsv = df[['image_id','annos']].copy() #create a csv of labels for fastai
labelcsv.iloc[:,1] = labelcsv['annos']\
.apply(lambda x: "".join(str(x))\
.replace(",","")\
.replace("[", "")\
.replace("]", ""))

In [50]:
labelcsv.head()

,image_id,annos
0,1,62 17 66 214 105 137 85
1,2,95 17 66 214 164 137 20 204 184
2,3,122 19 66 186 180 44 154 20
3,4,190 222 66 153 164 226 53 184
4,5,62 66 153 171 111 137 70 204 184


In [54]:
labelcsv.to_csv("/media/james/HDD/Data_sets/iMaterialist/train_labels.csv",index=False, header=False)

In [12]:
save_dir = "/media/james/HDD/Data_sets/iMaterialist/"

In [48]:
client = urllib3.PoolManager(500) #multiple requests
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [114]:
def DownloadImage(img_id, url):
    fname = os.path.join(save_dir, f'{img_id}.jpg')
    
    if os.path.exists(fname):
        return
    
    try:
        global client
        response = client.request('GET', url)
        img = response.data
    except: 
        print(f'download failed: {img_id}')
        
    try:
        pil_img = Image.open(BytesIO(img))
    except:
        print(f'PIL failed: {img_id}')
        
    try:
        img_rgb = pil_img.convert('RGB')
    except:
        print(f'RGB failed: {img_id}')
        
    try:
        img_rgb.save(fname, format='JPEG', quality=90)
    except:
        print(f'save failed: {img_id}')

In [49]:
iterlist = []
for row in df.itertuples():
    iterlist.append((row.image_id, row.image_url))

In [59]:
i = 0 

pool = multiprocessing.Pool(processes=12)
for _ in pool.starmap(DownloadImage, iterlist):
    i += 1
    if i % 1000 == 0:
        print(i)

1000
2000
3000
4000
5000
6000
7000
8000
9000


Process ForkPoolWorker-167:
Process ForkPoolWorker-165:
Process ForkPoolWorker-166:
Process ForkPoolWorker-168:
Process ForkPoolWorker-164:
Process ForkPoolWorker-157:
Process ForkPoolWorker-158:
Process ForkPoolWorker-160:
Process ForkPoolWorker-159:
Process ForkPoolWorker-162:
Process ForkPoolWorker-163:
Process ForkPoolWorker-161:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/process.py", line 

  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/media/james/HDD/Linux/Anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt


####  Download the test image set

In [ ]:
df = pd.read_json(f"{PATH}validation.json")

##### Downloading training images ~1 million (will take a long time)

The train file has metadata at its begining that prevents pandas from reading it.

In [108]:
file = '/media/james/HDD/Data_sets/iMaterialist/train.json'

js = json.load(open(file))

labeldf = pd.DataFrame(js['annotations'])
imagedf = pd.DataFrame(js['images'])
df = imagedf.merge(labeldf)
df.columns = ['imageId', 'image_url', 'labels']
df.head()

,imageId,image_url,labels
0,1,https://contestimg.wish.com/api/webimage/570f3...,"[95, 66, 137, 70, 20]"
1,2,https://contestimg.wish.com/api/webimage/5468f...,"[36, 66, 44, 214, 105, 133]"
2,3,https://contestimg.wish.com/api/webimage/54641...,"[170, 66, 97, 153, 105, 138]"
3,4,https://contestimg.wish.com/api/webimage/550b9...,"[18, 66, 44, 153, 164]"
4,5,https://contestimg.wish.com/api/webimage/54451...,"[189, 66, 176, 137, 153, 74, 181, 171, 138]"


In [113]:
df.shape

(1014544, 3)

In [109]:
save_dir = "/media/james/HDD/Data_sets/iMaterialist/train"

In [110]:
iterlist = []
for row in df.itertuples():
    iterlist.append((row.imageId, row.image_url))

In [116]:
i = 0 

pool = multiprocessing.Pool(processes=12)
for _ in pool.starmap(DownloadImage, iterlist):
    i += 1
    if i % 100000 == 0:
        print(i)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
